# Get play-by-play data for NFL games

One of the keys to performing any kind of data analysis is to have data in the first place (shocking, I know). For that task, the website Pro Sports Reference provides a great tool for querying huge databases of sports information. The user interface makes it possible to search for pretty much whatever kind of data you want. Football, basketball, college or pro, it's all there. But wouldn't it just be easier if you weren't limited to the website's interface? I think so. So I built a scraper to rebuild Pro Football Reference's ([PFR](www.profootballreference.com)) database of detailed play-by-play information on my own machine.
    
This notebook aims to strike a middle ground between a pedagogical walkthrough and code documentation. So I'll try and explain the overall picture of what the code aims to do and give extra attention to sections of the code that may not be clear at first glance.
    
## Disclaimer

As you read, keep in mind that there are certainly simpler and/or faster ways to accomplish what I'm trying to do. I don't claim to be an expert and this project was in large part a learning exercise.

## Basic structure

Rebuilding PFR's play-by-play database requires a few steps, which I will briefly outline here:
    
1. We need to know how to grab a table from the PFR website. I explain a bit about how to scrape the tables as I go along.
    
2. PFR has a lot of information. We need to know exactly which information we want, so that we can figure out where it is and grab it efficiently. I've separated the data gathering into two stages:
  1. For an individual team and season, get the information about each of their games that season.
  2. For each of those games, we can then get the relevant play-by-play information.

3. We need to put all of it into packages that we can easily access later.

## Initialization and Preamble
    
Import some packages that we'll be using and define some tables and dictionaries.

In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import time
import copy

Make a list of team names, including old ones going back to 1994 or so.

In [2]:
teams_long = [
    'Atlanta Falcons', 'New Orleans Saints', 'Carolina Panthers','Tampa Bay Buccaneers',
    'New York Giants', 'Dallas Cowboys', 'Philadelphia Eagles', 'Washington Redskins',
    'Minnesota Vikings', 'Green Bay Packers', 'Detroit Lions', 'Chicago Bears',
    'Seattle Seahawks', 'San Francisco 49ers', 'Arizona Cardinals',
    'St. Louis Rams', 'Los Angeles Rams',
    'Pittsburgh Steelers', 'Cleveland Browns', 'Cincinnati Bengals', 'Baltimore Ravens',
    'Buffalo Bills', 'New York Jets', 'Miami Dolphins', 'New England Patriots',
    'Tennessee Titans', 'Jacksonville Jaguars', 'Indianapolis Colts', 'Houston Texans',
    'Houston Oilers', 'Tennessee Oilers',
    'Denver Broncos', 'Oakland Raiders', 'San Diego Chargers', 'Kansas City Chiefs'
]

We also want a short-form unique identifier for each team. For PFR, these stay the same even when a franchise's branding changes. So the Tennessee Oilers and Tennessee Titans are both represented by 'oti'.

In [3]:
teams_short = [
    'atl','nor','car','tam','nyg','dal','phi','was',
    'min','gnb','det','chi','sea','sfo','crd','ram',
    'pit','cle','cin','rav','buf','nyj','mia','nwe',
    'oti','jax','clt','htx','den','rai','sdg','kan'
]

Define a dictionary to translate from the long-form to short-form team name.

In [4]:
shortform = {
    'Atlanta Falcons':'atl', 'New Orleans Saints':'nor', 
    'Carolina Panthers':'car', 'Tampa Bay Buccaneers':'tam',
    'New York Giants':'nyg', 'Dallas Cowboys':'dal',
    'Philadelphia Eagles':'phi', 'Washington Redskins':'was',
    'Minnesota Vikings':'min', 'Green Bay Packers':'gnb',
    'Detroit Lions':'det', 'Chicago Bears':'chi',
    'Seattle Seahawks':'sea', 'San Francisco 49ers':'sfo',
    'Arizona Cardinals':'crd', 'Los Angeles Rams':'ram', 'St. Louis Rams':'ram',
    'Pittsburgh Steelers':'pit', 'Cleveland Browns':'cle',
    'Cincinnati Bengals':'cin', 'Baltimore Ravens':'rav',
    'Buffalo Bills':'buf', 'New York Jets':'nyj',
    'Miami Dolphins':'mia', 'New England Patriots':'nwe',
    'Tennessee Titans':'oti', 'Jacksonville Jaguars':'jax', 
    'Houston Oilers':'oti', 'Tennessee Oilers':'oti',
    'Indianapolis Colts':'clt', 'Houston Texans':'htx',
    'Denver Broncos':'den', 'Oakland Raiders':'rai',
    'San Diego Chargers':'sdg', 'Kansas City Chiefs':'kan'
}

Now define a dictionary to get number form of months, which will be useful for making urls for individual games.

In [5]:
month = {
    'January':'01','February':'02','March':'03','April':'04','May':'05','June':'06',
    'July':'07','August':'08','September':'09','October':'10','November':'11','December':'12'
}

## Build functions to parse HTML

We need to be able to deal with raw HTML, which BeautifulSoup can do. This function will take a url and return a soup object containing the HTML.

In [6]:
def make_soup(url):
    res = requests.get(url)
    soup = BeautifulSoup(res.text,'lxml')
    return soup

The urls at PFR are formulaic, so we can write a short function to generate the url for a page describing a team's season for a given year

In [7]:
def season_url(team,year):
    base_url = 'http://www.pro-football-reference.com/teams'
    url = '{0}/{1}/{2}.htm'.format(base_url, team, year)
    return url

Let's take a look at what we're able to pull out so far.

In [8]:
soup = make_soup( season_url("cin",2015) )

parsed = soup.findAll(
    'div', {
        'class': 'table_wrapper',
        'id': 'all_games'
    })

print(parsed)

[<div class="table_wrapper" id="all_games">
<div class="section_heading">
<span class="section_anchor" data-label="Schedule &amp; Game Results" id="games_link"></span>
<h2>Schedule &amp; Game Results</h2> <div class="section_heading_text">
<ul>
</ul>
</div>
</div> <div class="table_outer_container">
<div class="overthrow table_container" id="div_games">
<table class="sortable stats_table" data-cols-to-freeze="1" id="games"><caption>Schedule &amp; Game Results Table</caption>
<colgroup><col></col><col></col><col></col><col></col><col></col><col></col><col></col><col></col><col></col><col></col><col></col><col></col><col></col><col></col><col></col><col></col><col></col><col></col><col></col><col></col><col></col><col></col><col></col><col></col><col></col></colgroup>
<thead>
<tr class="over_header">
<th aria-label="" class=" over_header " colspan="10" data-stat=""></th>
<th aria-label="" class=" over_header center" colspan="2" data-stat="header_score">Score</th>
<th aria-label="" class=

This is a bit scary to look at, but BeautifulSoup can help us out quite a bit.

The next function first reads the rows of the table and then picks out the header. Once we have the header and each individual row, we can make a pandas dataframe from our table. The dataframes make the data much easier to handle for bulk operations and we'll want to turn tables into dataframes wherever possible.

In [9]:
def make_season_df(team, year):

    # Make soup object for the team's season page
    soup = make_soup( season_url( team, year ) )
    
    # Pull out the table with the schedule
    try:
        parsed = soup.findAll(
            'div', {
                'class': 'table_wrapper',
                'id': 'all_games'
            })
    except:
        return 0
    
    
    # From that table, pull out the rows
    try:
        rows = parsed[0].findAll('tr')
    except:
        return 0
    
    # Separate the list of rows into their constituent elements,
    # resulting in a 2-dimensional array
    season_data = [
        [td.getText() for td in rows[i].findAll(['td','th'])] for i in range(len(rows))
    ]
    
    
    # Re-write a couple column titles in forms pandas can handle
    season_data[1][3:6] = ['Time','boxscore','result']
    season_data[1][8] = 'homeaway'
    season_data[1][10] = 'tm_score'
    season_data[1][11] = 'opp_score'
    
    # And finally make the dataframe
    season_df = pd.DataFrame( season_data[2:], columns=season_data[1] )
    
    return season_df

Let's check what this returns by looking at Carolina's 2015 season.

In [10]:
df = make_season_df( "car", 2015 )
df.head(5)

,Week,Day,Date,Time,boxscore,result,OT,Rec,homeaway,Opp,...,RushY,TO,1stD,TotYd,PassY,RushY,TO,Offense,Defense,Sp. Tms
0,1,Sun,September 13,1:03PM ET,boxscore,W,,1-0,@,Jacksonville Jaguars,...,105,1,18,265,169,96,3,-5.98,18.45,0.05
1,2,Sun,September 20,1:02PM ET,boxscore,W,,2-0,,Houston Texans,...,172,1,21,300,239,61,1,2.49,7.98,-3.79
2,3,Sun,September 27,1:03PM ET,boxscore,W,,3-0,,New Orleans Saints,...,119,,27,380,310,70,2,20.67,-9.37,-5.11
3,4,Sun,October 4,1:03PM ET,boxscore,W,,4-0,@,Tampa Bay Buccaneers,...,133,1,25,411,270,141,5,3.28,4.42,6.73
4,5,,,,,,,,,Bye Week,...,,,,,,,,,,


That looks pretty good! All of the source columns are present. All that's left to do is add which team and season the table is for so that we can recognize them once we put a bunch of these dataframes together.

Once we've done that, we want to figure out where to find the play-by-play data for each of these games. For PFR, this requires getting the date of the game and the short-form name of the home team. I get it for each game by applying a function to each row in the dataframe. The way it's written here, that requires creating a series for each row, which is not particularly efficient.

In [11]:
def contextualized_season(season_df, team, year):
    # Make a column for the season year and team
    season_df['team'] = team
    season_df['season'] = year

    # Make the url segment where we can find the box score for this game
    def make_boxscore_url(game,team,year):
        try:
            mn, day = str(game.Date).split()
            mn = month[mn]
            day = day.zfill(2)  # fill rest of string length 2 with zeros
            for tm in teams_long:
                if tm in str(game.Opp):
                    opponent = shortform[ tm ]
                    break
            location = team
            
            if int(mn) <= 3:
                year = int(year)+1
            if "@" in str(game.homeaway):
                location = opponent
                
            url_seg = "{0}{1}{2}0{3}".format(year,mn,day,location)
            return url_seg
        
        except:
            return "bye"

    season_df['box_url'] = season_df.apply( 
                    lambda row: make_boxscore_url(row,team,year), 
                    axis=1 )
    
    return season_df

So now we can put all of these little pieces together.

In [12]:
def grab_season(team, year):
    
    df = make_season_df( team, year )
    
    # A failure above will result in returning 0 for df.
    # Allow for failed grab without crashing loops later on
    if not type(df) is int:
        df = contextualized_season( df, team, year )
    
    return df

And we can loop over that for each team and season of interest. For the purpose of building the play-by-play dataset, PFR has detailed play-by-play data for most games 1995 and later, so the code here can be easily modified to loop over each team for each year since 1995.

In [13]:
season_list = []

for yr in range(1996,2016):
    for tm in teams_short:
        print(str(tm)+" "+str(yr))
        
        # Try to make a dataframe for this team this season
        season_df = grab_season( tm, yr )
        
        # If that was successful, add it to the list of dataframes
        if type(season_df) is int:
            pass
        else:
            season_list.append(season_df)
            
        # Wait between pings so we don't get blocked
        time.sleep(1)
        
# And concatenate all of our dataframes into one big table
games_df = pd.concat( season_list )

atl 1996
nor 1996
car 1996
tam 1996
nyg 1996
dal 1996
phi 1996
was 1996
min 1996
gnb 1996
det 1996
chi 1996
sea 1996
sfo 1996
crd 1996
ram 1996
pit 1996
cle 1996
cin 1996
rav 1996
buf 1996
nyj 1996
mia 1996
nwe 1996
oti 1996
jax 1996
clt 1996
htx 1996
den 1996
rai 1996
sdg 1996
kan 1996
atl 1997
nor 1997
car 1997
tam 1997
nyg 1997
dal 1997
phi 1997
was 1997
min 1997
gnb 1997
det 1997
chi 1997
sea 1997
sfo 1997
crd 1997
ram 1997
pit 1997
cle 1997
cin 1997
rav 1997
buf 1997
nyj 1997
mia 1997
nwe 1997
oti 1997
jax 1997
clt 1997
htx 1997
den 1997
rai 1997
sdg 1997
kan 1997
atl 1998
nor 1998
car 1998
tam 1998
nyg 1998
dal 1998
phi 1998
was 1998
min 1998
gnb 1998
det 1998
chi 1998
sea 1998
sfo 1998
crd 1998
ram 1998
pit 1998
cle 1998
cin 1998
rav 1998
buf 1998
nyj 1998
mia 1998
nwe 1998
oti 1998
jax 1998
clt 1998
htx 1998
den 1998
rai 1998
sdg 1998
kan 1998
atl 1999
nor 1999
car 1999
tam 1999
nyg 1999
dal 1999
phi 1999
was 1999
min 1999
gnb 1999
det 1999
chi 1999
sea 1999
sfo 1999
crd 1999
r

In [14]:
games_df.to_csv("game_info.csv")

In [13]:
games_df = pd.read_csv("game_info.csv")

## Getting the play-by-play for a single game

Despite BeautifulSoup being pretty good at handling HTML, I had problems consistently parsing the play-by-play tables from PFR and wound up doing a lot of it "by hand." That is to say that the bits of code for reading the tables aren't very pretty, but they do work.

The first order of business is to read a table header. Given the relevant text from the HTML, this function will do just that. As a bonus, the header might tell us who the home and away teams are, which we'll need to use to figure out which team has the ball on each play.

In [14]:
def read_header(txt):
    """Tries to read a header from a given input string of HTML text
    
    Returns a tuple consisting of a list of column titles as well as
    the home and away teams"""
    
    header = []
    home_team = 0
    away_team = 0

    # Cycle through each chunk in the raw HTML text
    
    # Look at each section of txt
    for section in txt:
        # For each chunk in that section
        for chunk in section.split():
            
            # Check whether this is the name of a statistic
            # If so, add it to the header
            if "data-stat" in chunk:
                header.append( chunk.split("=")[1].strip('"') )
                
            # This may be home/away team info
            if "</th>" in chunk:
                # strip html tags from the chunk
                chars = chunk.replace("</th","").strip(">")
                
                if "pbp_score_aw" in header[len(header)-1]:
                    away_team = chars                
                elif "pbp_score_hm" in header[len(header)-1]:
                    home_team = chars
                    
            # Or maybe we've reached the end of the header
            elif "</thead>" in chunk:
                break
            elif "</tr>" in chunk:
                break
                
                
    return header, home_team, away_team

Now that we have the header, we'll want to also read the entries for the rest of the table.

The code gets a bit ugly here, but the main idea is this: within the tags denoting each table entry, the displayed content is the only text not contained within additional html tags. So what the code does is to separate and list each of the tags in a single row (corresponding to one play) and make a separate list for everything that fits in between those tags, giving us the content of the entries in the row.

In addition, each of these rows may contain tags indicating which players are involved in the play, whether the previous play resulted in a change of possession, and whether the play resulted in a score. So the code checks for those as well.

In [15]:
def read_table(soup, table):
    """
    Reads a given soup object to find a play-by-play table.
    
    Returns a pandas dataFrame containing the table contents.
    """
    # Differentiate if we're looking for play-by-play table or the spread
    if table == "pbp":
        tablename = "all_pbp"
    elif table == "gameinfo":
        tablename = "all_game_info"
    else:
        tablename = table
        
    
    # Extract the HTML for the "all_pbp" table
    table = soup.find(
        'div', {
            'id': tablename
        })
    
    # If there's no text here, just return 0
    try:
        txt = table.findAll(text=True)
    except:
        return 0
    
    
    # Read the table header directly
    header, home_team, away_team = read_header(txt)
    
    # Turn rows back into strings
    rows = []
    table_body = False
    rowstring = ""
    for section in txt:
        for i, chunk in enumerate(section.split()):
            if ("<tbody>" in chunk) or ("<tr" in chunk):
                table_body = True
            if table_body:
                rowstring += " "+chunk
                if "</tr" in chunk:
                    rows.append(rowstring)
                    rowstring = ""
            if ("</tbody>" in chunk) or ("</table>" in chunk):
                break
                
    # Find data from spaces between tags in each of the rows
    all_entries = []
    stat_names = []
    players_involved = []
    change_of_possession = []
    scoring_plays = []
    in_tag = False
    en_str = ""
    tag_str = ""
    for r, row_string in enumerate(rows):
    
        # Tag-ify the row
        row_en = []
        row_tags = []
        for i, c in enumerate(row_string):
            if c == '<':
                in_tag = True
                row_en.append(en_str)
                en_str = ""
            if not in_tag:
                en_str += c
            elif in_tag:
                tag_str += c
            if c == '>':
                in_tag = False
                row_tags.append(tag_str)
                tag_str = ""
            
        # Make a list of entries in the row
        row_entries = []
        for i, tag in enumerate(row_tags):
            entry = ""
            if ('<th' in tag or '<td' in tag):
                finished = False
                while not finished:
                    for j in range(i,len(row_tags)):
                        entry += row_en[j]
                        if ('/th>' in row_tags[j] or '/td' in row_tags[j]):
                            finished = True
                            break
                row_entries.append(entry)
            
        # Add these entries to the list of all entries
        all_entries.append(row_entries)
            
        # Make a list of players involved in the play
        names = []
        for i, tag in enumerate(row_tags):
            if 'href' in tag:
                names.append(row_en[i+1])
        players_involved.append(names)
        
        # Check for whether play has a change of possession
        change_of_possession.append(0)
        if 'class="divider"' in row_string:
            # Previous play had a change of possession
            change_of_possession[r-1] = 1
            
        # Check for scoring plays
        if 'class=" score"' in row_string:
            scoring_plays.append(1)
        else:
            scoring_plays.append(0)
            
        
        # Make a new header to compare with one from above
        if r == 2:  #this should work but may need to change eventually
            for tag in row_tags:
                for term in tag.split():
                    if "data-stat" in term:
                        stat_names.append( term.split("=")[1].strip('"') )
                  
                
    if tablename == "all_pbp":
        # Add column for players involved
        header.append("players_involved")
        header.append("scoring_play")
        header.append("change_of_poss")
        stat_names.append("players_involved")
        stat_names.append("scoring_play")
        stat_names.append("change_of_poss")
        for i, play in enumerate(all_entries):
            play.append(", ".join(players_involved[i]))
            play.append(scoring_plays[i])
            play.append(change_of_possession[i])
            
        
    # Make a pandas DataFrame from the header and entries
    try:
        df = pd.DataFrame( all_entries, columns=header )
    except:
        df = pd.DataFrame( all_entries, columns=stat_names )
        
    
    # Add home and away team to dataframe
    if type(home_team) is not int:
        df['home_team'] = home_team
        df['away_team'] = away_team
    
        
    return df

If there's anywhere in this notebook that could be duplicated much more easily/simply, this is it. The source code for the detailed play-by-play table has a few extra fancy features that mean we can't parse them the same way as we did the team/year schedule tables. I've read that the HTML5lib module should be able to handle the sort of table that we're looking at, so that's a place to start simplifying the above code.

But if it works, we can move forward without worrying too much about how pretty the code is. So let's check that it all works.

First, write a function to complete the boxscore url from whence to grab play-by-play data, then try reading that table for a particular game. In this case, look at the first 10 rows in the play-by-play table for the Bengals-Steelers game on December 13, 2015.

In [16]:
def boxscore_url(url):
    base_url = "http://www.pro-football-reference.com/boxscores"
    whole_url = "{0}/{1}.htm".format(base_url,url)
    return whole_url

In [17]:
pbp_soup = make_soup(boxscore_url("201512130cin"))
pbp_df = read_table(pbp_soup,"pbp")

In [18]:
pbp_df.head(10)

,quarter,qtr_time_remain,down,yds_to_go,location,detail,pbp_score_aw,pbp_score_hm,exp_pts_before,exp_pts_after,home_wp,players_involved,scoring_play,change_of_poss,home_team,away_team
0,Quarter,Time,Down,ToGo,Location,Detail,PIT,CIN,EPB,EPA,Win%,,0.0,0.0,CIN,PIT
1,1st Quarter,,0,0,None,None,None,None,None,None,None,None,NaN,NaN,CIN,PIT
2,1,15:00,,,CIN 35,"Mike Nugent kicks off 65 yards, touchback",0,0,0,-0.28,56.6,Mike Nugent,0.0,1.0,CIN,PIT
3,1,15:00,1,10,PIT 20,DeAngelo Williams middle for 2 yards (tackle b...,0,0,0.28,0,57.4,"DeAngelo Williams, Vontaze Burfict",0.0,0.0,CIN,PIT
4,1,14:15,2,8,PIT 22,Ben Roethlisberger pass complete short left to...,0,0,0,0.23,56.6,"Ben Roethlisberger, Heath Miller, Vincent Rey,...",0.0,0.0,CIN,PIT
5,1,13:39,3,1,PIT 29,Ben Roethlisberger pass complete deep right to...,0,0,0.23,2.92,48.6,"Ben Roethlisberger, Markus Wheaton",0.0,0.0,CIN,PIT
6,1,13:31,1,10,CIN 40,DeAngelo Williams left guard for 3 yards (tack...,0,0,2.92,2.78,49.0,"DeAngelo Williams, Vontaze Burfict",0.0,0.0,CIN,PIT
7,1,12:47,2,7,CIN 37,Ben Roethlisberger pass complete short left to...,0,0,2.78,2.08,51.0,"Ben Roethlisberger, Martavis Bryant",0.0,0.0,CIN,PIT
8,1,12:05,3,7,CIN 37,Ben Roethlisberger pass complete short left to...,0,0,2.08,3.78,45.7,"Ben Roethlisberger, Heath Miller, Emmanuel Lamur",0.0,0.0,CIN,PIT
9,1,11:54,1,10,CIN 27,DeAngelo Williams right end for no gain (tackl...,0,0,3.78,3.23,47.4,"DeAngelo Williams, Michael Johnson",0.0,0.0,CIN,PIT


Also, I've built the above function with enough versatility that it can pick out whichever table from the webpage that we want, assuming we know what it's called. For instance, I'll try pulling out the table containing the Vegas spread.

In [19]:
spread_df = read_table(pbp_soup,"gameinfo")
print(spread_df)

         info                                               stat
0   Game Info                                               None
1    Won Toss                                           Steelers
2        Roof                                           outdoors
3     Surface                                         fieldturf 
4     Weather  63 degrees relative humidity 79%, wind 8 mph, ...
5  Vegas Line                            Cincinnati Bengals -2.5
6  Over/Under                                        50.0 (over)


In [20]:
sp_df = spread_df[ spread_df['info'].str.contains("Vegas") ]
print(sp_df.values[0,1])
#spread = spread_df.ix[5,'stat']

Cincinnati Bengals -2.5


We'll use this later to include the predicted spread in the information for this game

## Clean/Massage the data a little bit

Since the data will eventually be used, it's going to need a little bit of manipulation before that can happen. For instance, it might be useful for the row representing each play to be completely self-contained so that it doesn't rely on any other rows for context.

To see why that matters, consider trying to use the game situation before a given play to try and predict which team will win the game. The row contains information for the quarter, time remaining, down and distance and field position as well as some other things, but doesn't say which team has the ball. We would want to know that before trying to predict a winner, especially close to the end of the game.

The scores given in the PFR tables are also after the play has been completed, where our prediction would be based on the score before the play. The score before the play can be grabbed from the row for the previous play, but then each is no longer self-contained, so considerations like that are important before we try and do too much with our data.

In any case, one of the first things to do is drop rows that aren't associated with plays. One way to do that is to make a list of which rows satisfy a given condition and then filter the dataframe accordingly.

In [21]:
def drop_nonplays(df):
    
    # Take out rows for the start of quarters/OT
    bool_list = [ not i for i in df['quarter'].str.contains("Quarter") ]
    df = df.loc[ bool_list ]
    
    bool_list = [ not i for i in df['quarter'].str.contains("End") ]
    df = df.loc[ bool_list ]
    
    bool_list = [ not i for i in df['quarter'].str.contains("Overtime") ]
    df = df.loc[ bool_list ]
    
    # Take out rows for Coach's challenges and timeouts
    bool_list = [ not i for i in df['detail'].str.contains("challenged") ]
    df = df.loc[ bool_list ]
    
    bool_list = [ not i for i in df['detail'].str.contains("Timeout") ]
    df = df.loc[ bool_list ]
    
    return df

Now that all of the remaining rows signify plays, we might be able to figure out who has the ball for each play. I found this to be a surprisingly tricky problem to solve since the table contains very little information about which team is which. 

But we *do* know the score for the home and away team after each play, so we should be able to tell which team had possession on that play, and we *do* know when the ball changes hands. So unless the game is a scoreless tie, we can assign possession by figuring out who scored first and working in either direction to the beginning or end of the game from there.

But there's a catch! We can't simply assume that the offense scored the first points in the game; it could have been the defense or special teams. So we'll need to check the play details to see whether the scoring team had possession at the beginning of the play. The function below does that by checking the play details for a few trigger terms that most likely mean that the defense or punt/kickoff return team scored. This method isn't foolproof, but should be correct in almost all cases.

In [22]:
def assign_possession(df):
    """
    Return a list of whether home/away team has possession for each play
    """
    
    # Make sure columns are integers i.e. mathematically comparable
    df['scoring_play'] = df['scoring_play'].astype(int)
    df['pbp_score_aw'] = df['pbp_score_aw'].astype(int)
    df['pbp_score_hm'] = df['pbp_score_hm'].astype(int)
    
    # Set up some lists/arrays to iterate through
    scores = df['scoring_play'].values
    change_of_poss = df['change_of_poss'].values
    details = [str(i) for i in df['detail'].values]
    possession = ["xxx" for i in scores]
    
    # Get i for which ith row is the first score
    # This returns the index of the first instance of 1 in a numpy array
    first_score_idx = scores.tolist().index(1)
    
    
    # Check if there are triggers indicating defensive scores on the play
    triggers = ["fumbles", "intercepted", "kicks off", "punts ", " safety"]
    num_triggers = sum(
            [ 1 for s in triggers if s in details[first_score_idx] ]
        )
    
    # If home team scored first
    if df['pbp_score_hm'].values[first_score_idx] != 0:
        if num_triggers != 0:
            # First score was very probably by defense/return team
            possession[first_score_idx] = "away"
        else:
            possession[first_score_idx] = "home"
    
    else: # Away team scored first
        if num_triggers != 0:
            # First score was very probably by defense/return team
            possession[first_score_idx] = "home"
        else:
            possession[first_score_idx] = "away"
            
            
    # For plays prior to the first score,
    # trace possession arrow to the start of the game
    for p in range(first_score_idx):
        known_play = first_score_idx - p
        prev_play = first_score_idx - (p+1)
        
        if change_of_poss[prev_play] == 0: # No change of possession
            possession[prev_play] = possession[known_play]
            
        # otherwise, ball changed hands
        elif possession[known_play] == "home":
            possession[prev_play] = "away"
        elif possession[known_play] == "away":
            possession[prev_play] = "home"

            
    # Now do the same from the first score to end of game
    for p in range( first_score_idx, len(scores)-1 ):
        
        if change_of_poss[p] == 0: # No change of possession
            possession[p+1] = possession[p]
            
        elif possession[p] == "home":
            possession[p+1] = "away"
        elif possession[p] == "away":
            possession[p+1] = "home"
        
    
    return possession

In [23]:
df = drop_nonplays(pbp_df)
poss_list = assign_possession(df)
print(poss_list[:25])

['home', 'away', 'away', 'away', 'away', 'away', 'away', 'away', 'away', 'away', 'away', 'away', 'away', 'away', 'home', 'home', 'home', 'home', 'home', 'home', 'home', 'home', 'home', 'away', 'away']


I also mentioned getting the score before each play. We can get that by simply grabbing the score from the previous play.

In [24]:
def get_score_before_plays(df):
    
    # Get scores from the previous plays
    df['home_score'] = df.pbp_score_hm.shift(1)
    df['away_score'] = df.pbp_score_aw.shift(1)

    # Still need to add score 0-0 for the first play
    df['home_score'].iloc[0] = 0
    df['away_score'].iloc[0] = 0

In the interest of giving each play context, it may also be useful to add a column for which team wins the game.

In [25]:
def get_winner(df):
    """Return whether the home/away team wins the game"""
    last_play = df.tail(1)
    score_diff = int(last_play.pbp_score_hm) - int(last_play.pbp_score_aw)
    if score_diff > 0:
        return "home"
    elif score_diff < 0:
        return "away"
    else:
        return "tie"

There are also a couple of things that I want to do in order to prepare the data for use in a later model. These are not necessary for making the data meaningful and I won't describe them in detail, but the basic idea is to rewrite the stats from the offense's point of view. For instance, I'll make a new column for field position, ranging from -50, where the offense is at their own goal line, to 50, where the offense is an inch from scoring a touchdown.

In [26]:
def get_secs_played(play):
    try:
        if "OT" in str(play.quarter):
            qtr = 5
        else:
            qtr = int(play.quarter)
    except:
        qtr = 5
        
    try:
        mins_rem, secs_rem = str(play.qtr_time_remain).split(":")
    except:
        mins_rem = 15
        secs_rem = 0
        
    secs_played = 900*(qtr-1) + 900 - ( 60*int(mins_rem) + int(secs_rem) )
    return secs_played

In [27]:
def get_fieldpos_prefixes(df):
    sides = []
    fieldpos = [str(i) for i in df['location']]
    fieldpos = [i.split()[0] for i in fieldpos if i != '']
    for entry in fieldpos:
        if not entry in sides:
            sides.append(entry)
    
    if str(df['possession'].iloc[0]) == 'home':
        hm_side = sides[0]
        aw_side = sides[1]
    else:
        hm_side = sides[1]
        aw_side = sides[0]
        
    return hm_side, aw_side

In [28]:
def get_off_fieldpos(play, hm_side, aw_side):
    try:
        side, yardline = str(play.location).split()
        yardline = int(yardline)
        if str(play.possession) == "home":
            if side == hm_side:
                return -(50-yardline)
            elif side == aw_side:
                return 50-yardline
        elif str(play.possession) == "away":
            if side == hm_side:
                return 50-yardline
            elif side == aw_side:
                return -(50-yardline)
        return 0
    except:
        return 0

Now, finally, put all of the individual data cleaning functions together in one function.

In [29]:
def clean_game(df):
    
    # Drop non-play rows
    df = drop_nonplays(df)
    
    # Add column for possession
    df['possession'] = assign_possession(df)
        
    # Add column for the winning team
    df['winner'] = get_winner(df)
    
    # Add columns for score before plays happen
    get_score_before_plays(df)
    
    # Also rename pre-existing score columns
    df.rename( index=str, inplace=True, columns={
            "pbp_score_hm":"home_score_after",
            "pbp_score_aw":"away_score_after"})
    
    # Add column for seconds played
    df['secs_played'] = df.apply( 
        lambda row: get_secs_played(row),
        axis=1)
    
    # Add column for re-forumlated offensive field position
    hm_side, aw_side = get_fieldpos_prefixes(df)
    df['off_fieldpos'] = df.apply( 
        lambda row: get_off_fieldpos(row, hm_side, aw_side ),
        axis=1)
    
    return df

We should probably try using this to show that it all works properly.

In [30]:
pbp_df = clean_game(pbp_df)
pbp_df.head(10)

/home/evan/anaconda3/lib/python3.5/site-packages/pandas/core/indexing.py:132: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


,quarter,qtr_time_remain,down,yds_to_go,location,detail,away_score_after,home_score_after,exp_pts_before,exp_pts_after,...,scoring_play,change_of_poss,home_team,away_team,possession,winner,home_score,away_score,secs_played,off_fieldpos
2,1,15:00,,,CIN 35,"Mike Nugent kicks off 65 yards, touchback",0,0,0,-0.28,...,0,1.0,CIN,PIT,home,away,0.0,0.0,0,-15
3,1,15:00,1,10,PIT 20,DeAngelo Williams middle for 2 yards (tackle b...,0,0,0.28,0,...,0,0.0,CIN,PIT,away,away,0.0,0.0,0,-30
4,1,14:15,2,8,PIT 22,Ben Roethlisberger pass complete short left to...,0,0,0,0.23,...,0,0.0,CIN,PIT,away,away,0.0,0.0,45,-28
5,1,13:39,3,1,PIT 29,Ben Roethlisberger pass complete deep right to...,0,0,0.23,2.92,...,0,0.0,CIN,PIT,away,away,0.0,0.0,81,-21
6,1,13:31,1,10,CIN 40,DeAngelo Williams left guard for 3 yards (tack...,0,0,2.92,2.78,...,0,0.0,CIN,PIT,away,away,0.0,0.0,89,10
7,1,12:47,2,7,CIN 37,Ben Roethlisberger pass complete short left to...,0,0,2.78,2.08,...,0,0.0,CIN,PIT,away,away,0.0,0.0,133,13
8,1,12:05,3,7,CIN 37,Ben Roethlisberger pass complete short left to...,0,0,2.08,3.78,...,0,0.0,CIN,PIT,away,away,0.0,0.0,175,13
9,1,11:54,1,10,CIN 27,DeAngelo Williams right end for no gain (tackl...,0,0,3.78,3.23,...,0,0.0,CIN,PIT,away,away,0.0,0.0,186,23
10,1,11:21,2,10,CIN 27,DeAngelo Williams left tackle for 7 yards (tac...,0,0,3.23,3.47,...,0,0.0,CIN,PIT,away,away,0.0,0.0,219,23
12,1,10:34,3,3,CIN 20,Ben Roethlisberger pass complete short middle ...,0,0,3.47,6.97,...,0,0.0,CIN,PIT,away,away,0.0,0.0,266,30


## Construct a loop to grab plays for many games

Since that seems to work, the next step is to try and grab the same table from many pages. We can do this in generally the same way we looped over teams and seasons before; the key here is to get a list of games to loop over. 

Conveniently (necessarily), we already have that list from before, we just have to access it.

In [31]:
# Get info for who's playing in each game and what the boxscore url is
game_list = [ 
        str(row.box_url) for i, row in games_df.iterrows()
    ]

print(game_list[:5])

['199609010car', '199609080atl', 'bye', '199609220atl', '199609290sfo']


We can loop through this list and grab the play-by-play data for each of these games. But we don't want to duplicate effort and each of these games is listed twice, so keep track of which urls have been used already.

In [38]:
# Arbitrarily limit the loop
games_to_search = game_list[:]

# List for play-by-play dataframes
pbp_list = []

# Keep track of which urls have been used already
urls_visited = []
for url in games_to_search:
    if (url not in urls_visited) and ('bye' not in url):
        
        # We haven't tried to find this game yet
        urls_visited.append(url)
        
        # Try to grab the game's play-by-play data
        print("Looking up "+url)
        pbp_soup = make_soup(boxscore_url(url))
        pbp_df = read_table(pbp_soup, "pbp")
        
        # Try to grab the game's spread
        spread_df = read_table(pbp_soup, "gameinfo")
        
        # If that was successful, clean the resulting dataframe
        if type(pbp_df) is int:
            print("Play-by-play Lookup unsuccessful")
        elif type(spread_df) is int:
            print("Spread Lookup unsuccessful")
        
        else:
            pbp_df = clean_game(pbp_df)
            
            # Pull out the spread
            spread_df = spread_df[ spread_df['info'].str.contains("Vegas") ]
            spread = spread_df.values[0,1]
            
            # Also make sure to keep track of game's unique id and spread
            pbp_df['game_url'] = url
            pbp_df['spread'] = spread
            
            # Add this dataframe to the list of individual game dfs
            pbp_list.append(pbp_df)
            
        # Wait a second before pinging again
        time.sleep(1)
        

# Make one big dataframe from all of the little ones
all_plays_df = pd.concat(pbp_list)
all_plays_df.to_csv("pbp_data.csv")

Looking up 199609010car


/home/evan/anaconda3/lib/python3.5/site-packages/pandas/core/indexing.py:132: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


Looking up 199609080atl
Looking up 199609220atl
Looking up 199609290sfo
Looking up 199610060det
Looking up 199610130atl
Looking up 199610200dal
Looking up 199610270atl
Looking up 199611030atl
Looking up 199611100ram
Looking up 199611170atl
Looking up 199611240cin
Looking up 199612020atl
Looking up 199612080nor
Looking up 199612150atl
Looking up 199612220jax
Looking up 199609010sfo
Looking up 199609080nor
Looking up 199609150cin
Looking up 199609220nor
Looking up 199609290rav
Looking up 199610060nor
Looking up 199610130nor
Looking up 199610200car
Looking up 199611030nor
Looking up 199611100nor
Looking up 199611240tam
Looking up 199612010nor
Looking up 199612150nyg
Looking up 199612210ram
Looking up 199609220car
Looking up 199609290jax
Looking up 199610060min
Looking up 199610130car
Looking up 199610270phi
Looking up 199611100car
Looking up 199611170ram
Looking up 199611240oti
Looking up 199612010car
Looking up 199612080sfo
Looking up 199612150car
Looking up 199612220car
Looking up 19970

In [40]:
all_plays_df['spread'].unique()[:15]

array(['Atlanta Falcons 3.0', 'Atlanta Falcons -4.0',
       'Atlanta Falcons -2.0', 'San Francisco 49ers -13.0',
       'Detroit Lions -10.0', 'Atlanta Falcons -1.5',
       'Dallas Cowboys -15.0', 'Pittsburgh Steelers 6.0',
       'Carolina Panthers 1.5', 'Atlanta Falcons 1.5',
       'Cincinnati Bengals -6.0', 'San Francisco 49ers 9.5',
       'New Orleans Saints -2.0', 'Atlanta Falcons -5.5',
       'Jacksonville Jaguars -8.5'], dtype=object)

## That's all there is to it!

Thanks for sticking with me. It took a bit of work, but now we've got a .csv file of NFL play-by-play data for four games in 1998. And by modifying the appropriate loops just a little bit, you can scrape yourself a whole lot more.